In [ ]:
import os
import cv2
import dlib
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def create_folders(crop_folder):
    """크롭 폴더 구조를 생성합니다."""
    os.makedirs(crop_folder, exist_ok=True)

def read_image_in_memory(image_path):
    """이미지 파일을 메모리로 읽어들입니다."""
    with open(image_path, 'rb') as f:
        file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
        frame = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    return image_path, frame

def process_image(image_data, crop_folder, class_id):
    try:
        image_path, frame = image_data

        # dlib의 강아지 얼굴 탐지 모델 로드
        detector = dlib.cnn_face_detection_model_v1('dogHeadDetector.dat')

        # 강아지 얼굴 탐지
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detections = detector(gray, 1)

        # 얼굴이 검출된 경우만 저장
        if detections:
            orig_height, orig_width = frame.shape[:2]

            for i, d in enumerate(detections):
                left, top, right, bottom, _ = (d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence)

                # 좌표가 이미지 범위를 벗어나지 않도록 조정
                left = max(0, left)
                top = max(0, top)
                right = min(orig_width, right)
                bottom = min(orig_height, bottom)

                # 박스에 맞게 이미지 크롭
                cropped_img = frame[top:bottom, left:right]

                # 크롭된 이미지가 유효한지 확인
                if cropped_img.size == 0:
                    print(f"Warning: Cropped image is empty. Skipping this detection.")
                    continue

                # 크롭된 이미지 리사이즈 (224x224)
                resized_cropped_img = cv2.resize(cropped_img, (224, 224))

                # 개별 클래스 폴더 생성
                class_folder = os.path.join(crop_folder, str(class_id))
                os.makedirs(class_folder, exist_ok=True)

                # 크롭된 이미지 저장 경로 설정
                crop_image_name = f'{os.path.basename(image_path)}'
                crop_image_path = os.path.join(class_folder, crop_image_name)

                # 크롭된 이미지 저장
                cv2.imwrite(crop_image_path, resized_cropped_img)
                print(f"Cropped and resized image saved as {crop_image_path}")

    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e)
    except RuntimeError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def process_all_images_in_folder(folder_path, crop_folder):
    """폴더 내 모든 이미지 파일을 처리합니다."""
    create_folders(crop_folder)
    
    image_paths = []
    for class_folder in os.listdir(folder_path):
        class_folder_path = os.path.join(folder_path, class_folder)
        if os.path.isdir(class_folder_path):
            for image_file in os.listdir(class_folder_path):
                if image_file.endswith('.jpg'):
                    image_path = os.path.join(class_folder_path, image_file)
                    image_paths.append((image_path, crop_folder, class_folder))

    # 이미지를 메모리로 읽어들이기
    with ThreadPoolExecutor() as executor:
        image_data_list = list(executor.map(read_image_in_memory, [image_path[0] for image_path in image_paths]))

    # 읽어들인 이미지 데이터를 사용하여 병렬 처리
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda p: process_image(p[0], p[1], p[2]), zip(image_data_list, [crop_folder]*len(image_data_list), [class_id for _, _, class_id in image_paths])))

folder_path = 'sample_data'
crop_folder = 'Dog_face_net_cropped_images'

process_all_images_in_folder(folder_path, crop_folder)
